In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
import gymnasium as gym
import os 

# env = gym.make('CartPole-v1',render_mode="rgb_array")

# episodes = 1000
# best = 0 
# for episode in range(1, episodes+1):
#     state = env.reset() 
#     done = False	
#     score = 0 

#     while not done:
#         # env.render()
#         action = env.action_space.sample()
#         n_state, reward, done,_, info = env.step(action) 
#         score += reward
    
#     if best <= score:
#         best = score
#     print('Episode:{}Score:{}'.format(episode, score))
# print(best)
# env.close() 


In [19]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
# import gymnasium as gym
import os, gym 
import random

from gym.spaces import Box, Dict, Discrete, MultiBinary, MultiDiscrete

class AEOSEnv(gym.Env):
    def __init__(self) -> None:
        self.action_space = Discrete(3)
        self.observation_space = Box(low=0, high=100,shape=(1,))
        self.state = 38+random.randint(-3,3)
        self.length = 60

    def step(self,action):
        self.state += action-1
        self.length -=1
        if self.state>=37 and self.state <=39:
            reward = 1
        else:
            reward = -1

        if self.length <=0:
            done = True
        else:
            done = False


        info ={}
        return self.state, reward, done, info
    def reset(self):
        self.length = 60
        self.state = 38+random.randint(-3,3)
        return self.state
    def render(self):
        pass



In [20]:
env = AEOSEnv()
episodes = 5
best = 0 
for episode in range(1, episodes+1):
    state = env.reset() 
    done = False	
    score = 0 

    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done ,info = env.step(action) 
        score += reward
    
    if best <= score:
        best = score
    print('Episode:{} Score:{}'.format(episode, score))
print(best)
env.close() 

Episode:1 Score:-22
Episode:2 Score:-16
Episode:3 Score:-30
Episode:4 Score:-8
Episode:5 Score:-60
0


In [21]:
log_path = os.path.join("Training",'Logs')
env = DummyVecEnv([lambda:env])
model = PPO("MlpPolicy",env, verbose=1,tensorboard_log=log_path)
model.learn(total_timesteps=20000) # 模型训练20000次

Using cpu device
Logging to Training\Logs\PPO_4


c:\Users\hithx\miniconda3\envs\drl\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


-----------------------------
| time/              |      |
|    fps             | 2424 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1648        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.011001111 |
|    clip_fraction        | 0.0555      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | -0.000299   |
|    learning_rate        | 0.0003      |
|    loss                 | 28.1        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.00374    |
|    value_loss           | 59.3        |
-----------------------------------------
----------------------------------

In [ ]:
log_path = os.path.join("Training",'Logs')
env = gym.make('CartPole-v1',render_mode="human")
env = DummyVecEnv([lambda:env])
model = PPO("MlpPolicy",env, verbose=1,tensorboard_log=log_path)
model.learn(total_timesteps=20000) # 模型训练20000次


In [ ]:
ppo_path = os.path.join('Training','Saved','ppo_cart')
model.save(ppo_path)

In [ ]:
del model

In [ ]:
model = PPO.load(ppo_path, env=env)

In [ ]:
evaluate_policy(model,env, n_eval_episodes=10,render=True)

In [22]:
import os 

log_path = os.path.join("Training",'Logs')

%load_ext tensorboard
%tensorboard --logdir=log_path

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 21172), started 20:50:47 ago. (Use '!kill 21172' to kill it.)

In [4]:
import gymnasium as gym
import numpy as np

from stable_baselines3 import DDPG
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
from stable_baselines3.common.env_util import make_atari_env
# env = gym.make("Pendulum-v1", render_mode="rgb_array")


env = make_atari_env("Skiing-v4", n_envs=4, seed=0)
# The noise objects for DDPG
n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

model = DDPG("MlpPolicy", env, action_noise=action_noise, verbose=1)
model.learn(total_timesteps=10000, log_interval=10)
model.save("ddpg_pendulum")
vec_env = model.get_env()

del model # remove to demonstrate saving and loading

model = DDPG.load("ddpg_pendulum")

obs = vec_env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = vec_env.step(action)
    env.render("human")

IndexError: tuple index out of range